# [IceBridge Alaska](https://www.nasa.gov/image-feature/operation-icebridge-exploring-alaska-s-mountain-glaciers) Altimetry Analysis Tools

#### Code Developers: Anthony Arendt, Evan Burgess, Chris Larsen, Christian Kienholz

These codes import the altimetry data from UAF to the RDS database.

## Overview

The Alaska altimetry code depends heavily on a backend PostgreSQL database for storing and analyzing mass balance trends. This originates from extensive use of PostgreSQL/PostGIS for generating the Alaska glacier inventory which eventually formed the basis for the Randolph Glacier Inventory. This notebook is used to create a new/manage and existing relational database storing the Alaska-altimetry products. This code can only be run with admin privliges.

### Database host location

We have experimented with hosting the database in several locations over the years. For some time it was served from a computer located in the University of Alaska Fairbanks glaciology group. Currently we host the database on an Amazon Web Services Relational Database System (RDS).

### Creating the database

Before running any of the code below, these steps are required:

* connect to the database with admin privilges
* create a new database called "Alaska-altimetry"
* in a SQL command window, enabl the PostGIS extension: ```CREATE EXTENSION postgis;```
* run new_lamb_table.sql located in "../sql" directory


### Managing users

* create role: 'admin' with 'postgres' priviliges (corresponds to "ice2oceansdb-admin" in env file): this user can run the codes below for generating new tables, etc.
* create role: 'altimetryuser' with login priviliges. Run: ```GRANT SELECT ON ALL TABLES IN SCHEMA public TO altimetryuser;``` ((corresponds to "ice2oceansdb-altimetryuser" in env file): this user represents general reserachers working with database queries


#### Import libraries, including custom AltPy tools

In [1]:
import yaml
import pandas as pd

import AltPy.Altimetry as alt
import AltPy.UpdateDb as updatedb

#### Read in credentials needed to connect to the ftp site storing the raw data, and the relational database storing analysis products

In [2]:
credentials = yaml.load(open('../.env.yaml'), Loader=yaml.FullLoader)

#### If importing new data, connect here to the ftp site at UAF

Start with a list of glaciers you want to acquire from UAF. Build this as a pandas dataframe.

In [3]:
# create some dummy data for now. This should be read in later.

data = [['Tsina', 'G214127E61264N'],['BlackRapids', 'G213513E63423N']]

# Create the pandas DataFrame 
glacierNames = pd.DataFrame(data, columns = ['name', 'glimsid']) 

tempDirectory = 'data/'

### upload all the data to the database

In [4]:
updatedb.get_ftp_data(credentials['UAF-ftp'], credentials['ice2oceansdb-admin'], 
                      glacierNames, tempDirectory)

'data successfully uploaded'

Some tests to make sure we can read back data ingested to the db

In [5]:
engine, cursor = alt.ConnectDb(credentials['ice2oceansdb-Altimetry-user'])

In [ ]:
cursor.execute('Select * from lamb')
testdata = cursor.fetchall()
print(testdata)
cursor.close()

# Building glacier geometry files

These altimetry scripts rely on glacier polygons from the [Randolph Glacier Inventory](https://www.glims.org/RGI/). Area and hyposemtric information from these polygons enable us to calculate glacier-wide mass balances from centerline altimetry profiles (for measured glaciers) and to extrapolate observations from measured to unmeasured glaciers.

Our process is to acquire the RGI data in our own database so that we can query it as needed for our anlysis. The ingest of RGI to the database only needs to be done once.

**NOTE**: these scripts were created when RGI 4.0 was the current version for Alaska, so we continue to use that version now. See [Issue #3](https://github.com/aaarendt/Alaska-altimetry/issues/3) for recommended upgrade to RGI 6.0.

## Importing the RGI shapefiles into the database

These are the steps for getting the RGI data into the database:

* download the Alaska RGI v4.0 from the [NSIDC/Global Land Ice Measurements from Space website]((https://www.glims.org/RGI/rgi40_dl.html)).
* connect to the database as administrator
* import the shapefile to a table called "modern" using one of several PostGIS extensions. For example, use QGIS (TODO: see [Issue #4](https://github.com/aaarendt/Alaska-altimetry/issues/4) for the need to also populate the table/column metadata).

## Generate the E-RGI table

The RGI delineates glaciers according to a hypsometric basin approach. There are a few cases within the altimetry project where it makes sense to treat some collection of glaciers as a single glacier. Or, we may wish to reassign the glacier type, for example Taku glacier which today is land terminating, but still essentially behaves like a tidewater glacier. Therefore we create a new table from the RGI called "ergi".

The script for generating table *ergi* is in '../sql/new_ergi.sql'. This script can be run manually in a SQL terminal once connected to the database (e.g. in pgAdmin or dBeaver). 

The *ergi* table depends on the *modern* table as well as a table called *modernadditional*. The additional information includes slope, aspect, max and min elevation. It comes from work by [Kienholz et al., 2015](https://www.igsoc.org/journal/61/227/t14j230.pdf). Contact us to get this table (it is currently not posted on any public portal).

## Generate the ergi_mat_view table

For convenience and efficiency, we created a [materialized view](https://en.wikipedia.org/wiki/Materialized_view) of the *ergi* table. To generate this, run the SQL code in '../sql/ergi_mat_view.sql'. This script requires two additional tables:

* *arendtregions* containing polygons describing the major mountain regions of Alaska as in [Arendt et al., 2002](https://science.sciencemag.org/content/297/5580/382).
* *burgessregions* a modification of the regions to accommodate analysis in [Larsen et al., 2015](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2015GL064349).

Contact us for access to these tables.





The LiDAR Altimetry data and glacier geometry data used in [Larsen et al.(2015)]() are stored in the following table structure.  The key point to understand is that here we do not start from the raw LiDAR point clouds. Instead, the table lamb  contains the surface elevation change rate profiles for each glacier over each possible interval. Each row was in this table was generated using a semi-manual step (discussed in Arendt et al., [2002] and Johnson et all. [2013]), where a user defines a bin size, a glacier polygon etc. and then runs a matlab script called 'lamb' to generate a top-bottom profile of surfaceelevation change rates. This script also outputs the along profile IQR of the measured surface elevation change and the mass balance integrated across the user defined glacier polygon.  All of this data is included in the lamb table. The only part of the lamb table used by Larsen et al., is the elevation change rate profile and the IQR for each glacier and the glacier geometry is extracted from the ergi table and ergi_mat_view materialized view.

The glacier geometry is provided by a modified version of the RGI 4.0 called the ergi.  The ergi has some differences that accomodate the UAF Altimetry flightlines.  Important glacier attributes including Glacier name, terminus type,surge-type,and region are all stored as points.  We then run a spatial join with the ergi to create a materialzed view of the ergi table with the appropriate attributes.  The ergi has also been split into 30 m elevation bins using the DEM described in Kienholz et al. [2015] to capture every glacier hypsometry in ergibins.
